# <center> **ПРОЕКТ**

### <center> Исследовать поведение пользователей в обновлённом приложении.

студент: Смирнов Алексей 

група: DAPR_197

## <center> Цель проекта проверить гипотезы:

1. Определить, насколько обучение сокращает время прохождения этапов игры.

2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.

3. Определить, насколько прозрачен процесс взаимодействия с игрой.



Важно! Выполнять задание необходимо на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

## <center> Подготовка данных

In [ ]:
import pandas as pd 
import numpy as np
import plotly
import plotly.express as px

events_df = pd.read_csv('Events.csv', sep=',')
purchase_df = pd.read_csv('Purchase.csv', sep=',')

events_copy = events_df.copy()
purchase_copy = purchase_df.copy()

# согласно заданию расчеты проводятся с пользователями зарегистрированными в 2018г.
# Создаем mask для фильтрайии и определяем список с пользователями
mask_1 = events_df['start_time'] < '2019-01-01'
mask_2 = events_df['start_time'] >= '2018-01-01'
mask_3 = events_df['event_type'] == 'registration'
user_list_2018 = events_df[mask_1 & mask_2 & mask_3]['user_id'].to_list()

# Создание DataFrame: events_df и purchase_df c пользователями зарегистрировавшимися в 2018г.
events_df = events_df[events_df.user_id.isin(user_list_2018)]
purchase_df = purchase_df[purchase_df.user_id.isin(user_list_2018)]

# Добавляем и переименовываем столбцы
purchase_df['event_type'] = 'purchase'
events_df = events_df.rename(columns ={'id':'event_id'})
purchase_df = purchase_df.rename(columns={"id": "purchase_id", "event_datetime": "start_time"})

# Дата в формате datetime
events_df['start_time']=pd.to_datetime(events_df['start_time'], errors='coerce')
purchase_df['start_time']=pd.to_datetime(purchase_df['start_time'])

# список пользователей прошедших обучение
tutorial_finish_list = list(events_df[events_df['event_type'] == 'tutorial_finish']['user_id'].unique())
# список пользователей начавших но незакончивших обучение
tutorial_start_list = list(events_df[(events_df['event_type'] == 'tutorial_start') & (
    ~events_df['user_id'].isin(tutorial_finish_list))]['user_id'].unique())

In [ ]:
#total_events_df - объединенная таблица (из events и purchase)
total_events_df = pd.concat([events_df, purchase_df], sort=False)

#удаляем старые индексы и сортируем по дате
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')


# Создадим сводную таблицу
pivot_df = total_events_df.pivot_table(
    values='start_time', index='user_id', columns='event_type', aggfunc='min')

# Cбросим индексы
pivot_df = pivot_df.reset_index()

pivot_df.head(5)

event_type,user_id,level_choice,pack_choice,purchase,registration,tutorial_finish,tutorial_start
0,27832,NaT,NaT,NaT,2018-01-01 03:48:40,NaT,NaT
1,27833,NaT,NaT,NaT,2018-01-01 04:07:25,2018-01-01 17:50:08,2018-01-01 17:47:40
2,27834,NaT,NaT,NaT,2018-01-01 08:35:10,2018-01-01 19:48:01,2018-01-01 19:46:11
3,27835,2018-01-01 20:37:22,2018-01-01 20:38:43,NaT,2018-01-01 11:54:47,2018-01-01 15:06:15,2018-01-01 15:00:51
4,27836,NaT,NaT,NaT,2018-01-01 13:28:07,2018-01-01 15:42:58,2018-01-01 14:54:40


## <center> Гипотеза 1

<center> Определить, насколько обучение сокращает время прохождения этапов игры.

In [ ]:
# Функция определяющая статус обучения для пользователя
def status(x, user_id):
    if x[user_id] in tutorial_finish_list:
        return 'finished'
    elif x[user_id] in tutorial_start_list:
        return 'not_finished'
    else:
        return 'not_start'

# Создаем признак показывающий в какую группу входит пользователь исходя из статуса его обучения прошел/начал но неокончил/не проходил
pivot_df['user_group'] = pivot_df.apply(lambda x: status(x, 'user_id'), axis=1)
pivot_df.head(5)

event_type,user_id,level_choice,pack_choice,purchase,registration,tutorial_finish,tutorial_start,user_group
0,27832,NaT,NaT,NaT,2018-01-01 03:48:40,NaT,NaT,not_start
1,27833,NaT,NaT,NaT,2018-01-01 04:07:25,2018-01-01 17:50:08,2018-01-01 17:47:40,finished
2,27834,NaT,NaT,NaT,2018-01-01 08:35:10,2018-01-01 19:48:01,2018-01-01 19:46:11,finished
3,27835,2018-01-01 20:37:22,2018-01-01 20:38:43,NaT,2018-01-01 11:54:47,2018-01-01 15:06:15,2018-01-01 15:00:51,finished
4,27836,NaT,NaT,NaT,2018-01-01 13:28:07,2018-01-01 15:42:58,2018-01-01 14:54:40,finished


In [ ]:
# Создаем новые признаки
"""
delta_start - Время между регистрацией и началом обучения
delta_finish - Время между началом обучения и концом обучения
delta_level_choice - Время между регистрацией и выбором уровня сложности
delta_pack_choice - Время между выбором уровня сложности и выбором пакета вопросов
delta_purchase - Время между выбором пакета вопросов и покупки платных вопросов
delta_registration-purchase - Время между регистрацией и покупкой платных вопросов
"""
pivot_df['delta_start'] = ((
    pivot_df['tutorial_start'] - pivot_df['registration'])/pd.Timedelta('1 minute')/60)
pivot_df['delta_finish'] = ((
    pivot_df['tutorial_finish'] - pivot_df['tutorial_start'])/pd.Timedelta('1 minute')/60)
pivot_df['delta_level_choice'] = ((
    pivot_df['level_choice'] - pivot_df['registration'])/pd.Timedelta('1 minute')/60)
pivot_df['delta_pack_choice'] = ((
    pivot_df['pack_choice'] - pivot_df['level_choice'])/pd.Timedelta('1 minute')/60)
pivot_df['delta_purchase'] = ((
    pivot_df['purchase'] - pivot_df['pack_choice'])/pd.Timedelta('1 minute')/60)
pivot_df['delta_registration-purchase'] = ((
    pivot_df['purchase'] - pivot_df['registration'])/pd.Timedelta('1 minute')/60)

pivot_df.head(5)

event_type,user_id,level_choice,pack_choice,purchase,registration,tutorial_finish,tutorial_start,user_group,delta_start,delta_finish,delta_level_choice,delta_pack_choice,delta_purchase,delta_registration-purchase
0,27832,NaT,NaT,NaT,2018-01-01 03:48:40,NaT,NaT,not_start,NaN,NaN,NaN,NaN,NaN,NaN
1,27833,NaT,NaT,NaT,2018-01-01 04:07:25,2018-01-01 17:50:08,2018-01-01 17:47:40,finished,13.670833,0.041111,NaN,NaN,NaN,NaN
2,27834,NaT,NaT,NaT,2018-01-01 08:35:10,2018-01-01 19:48:01,2018-01-01 19:46:11,finished,11.183611,0.030556,NaN,NaN,NaN,NaN
3,27835,2018-01-01 20:37:22,2018-01-01 20:38:43,NaT,2018-01-01 11:54:47,2018-01-01 15:06:15,2018-01-01 15:00:51,finished,3.101111,0.090000,8.709722,0.0225,NaN,NaN
4,27836,NaT,NaT,NaT,2018-01-01 13:28:07,2018-01-01 15:42:58,2018-01-01 14:54:40,finished,1.442500,0.805000,NaN,NaN,NaN,NaN


In [ ]:
# Групируем по группе пользователя и определяем среднее значение для времени между этапами
table_group = pivot_df.groupby('user_group')[[
    'delta_start', 'delta_finish', 'delta_level_choice', 'delta_pack_choice',
    'delta_purchase', 'delta_registration-purchase']].mean().round(2)

table_group

event_type,delta_start,delta_finish,delta_level_choice,delta_pack_choice,delta_purchase,delta_registration-purchase
user_group,,,,,,
finished,4.52,0.2,7.09,0.09,89.30,96.50
not_finished,5.42,NaN,8.22,0.08,93.25,101.55
not_start,NaN,NaN,5.26,0.09,100.83,105.10


In [ ]:
# Столбчатая диаграмм
fig_1 = px.bar(
    x= 'event_type',
    y= table_group.columns,
    color= table_group.index,
    facet_col=table_group.index,
    data_frame=table_group,
    orientation='v',
    labels = { "value" :  "время прохождения этапов в ч.", "event_type": "Тип события", "user_group": "Группа пользователей:"},
    height=500,
    width=1000,
    title="Среднее время между прохождения этапов игры по группам пользователей"
)
fig_1.show()

## <center> ВЫВОД

Из графика "Среднее время между прохождения этапов игры по группам пользователей" видно, что пользователи прошедшие обучение проходят этапы игры бастрее всех (в среднем от регистрации до покупки у них уходит 96.50 ч.). Помедленее это делают пользователи не закончившие обучение (в среднем от регистрации до покупки у них уходит 101.55 ч.) и медленее всех проходят этапы пользователи не проходившие обучение (в среднем от регистрации до покупки у них уходит 105.10 ч.). Исключением служат этапы выбора уровня сложности тут лучший результат показывают пользователи не проходившие обучение (возможно из за того что это их первое после регистрации взаимодействие с игрой) и этап выбора пакета вопросов где результаты равны. Из этого можно сказать что обучение влияет на время прохождения этапов игры. Более подробную информацию о времени прохождении этапов можно посмотреть в таблице: table_group 

## <center> Гипотеза 2

<center> Существует ли зависимость между вероятностью оплаты и количеством обучений, которые начинал или завершал пользователь?

*** В вопросе указано "количеством обучений, которые начинал ИЛИ завершал пользователь?" союз или подразумевает выполнение одного из перечисленных условий например если обучение было начато несколько раз но ниразу неокончино то его тоже надо включить в выборку. Я этого не делал и работал только с обучениями которые были завершины, так как завершенное обучение оказывают большее влияние на оплату, что я и показываю в первом коде после этого примечания. (в данном примере это не оказывает существенного влияния если при поиске количества обучений подставить 'tutorial_start' итоговые данные по проекту изменятся на 0,1%)

In [ ]:
# Доля групп от общего числа покупателей в %
group_from_all_user_purchase = round((
    pivot_df[~pivot_df['purchase'].isna()].groupby('user_group')['user_id'].nunique()/
    pivot_df[~pivot_df['purchase'].isna()]['user_id'].nunique())*100, 1)

group_from_all_user_purchase

user_group
finished        90.4
not_finished     8.2
not_start        1.4
Name: user_id, dtype: float64

In [ ]:
fig_2 = px.pie(values=group_from_all_user_purchase, 
              names=group_from_all_user_purchase.index,
              title='Доля групп от общего числа покупателей в %',
              height=400,
              width=600
              )
fig_2.show()

In [ ]:
# % пользователей совершивших покупку от обшего количества в группе
percent_of_users_made_a_purchase = pivot_df[~pivot_df['purchase'].isna()].groupby(
    'user_group')['user_id'].nunique()/pivot_df.groupby('user_group')['user_id'].nunique()

round(percent_of_users_made_a_purchase, 3)

user_group
finished        0.141
not_finished    0.081
not_start       0.003
Name: user_id, dtype: float64

In [ ]:
# Создаем сводную таблицу с кол. пройденого обучения (aggfunc='count')
tutorial_count_df = total_events_df.pivot_table(values='start_time', index= 'user_id', columns= 'event_type', aggfunc='count')
tutorial_count_df.head(5)

event_type,level_choice,pack_choice,purchase,registration,tutorial_finish,tutorial_start
user_id,,,,,,
27832,NaN,NaN,NaN,1.0,NaN,NaN
27833,NaN,NaN,NaN,1.0,1.0,1.0
27834,NaN,NaN,NaN,1.0,1.0,1.0
27835,1.0,1.0,NaN,1.0,1.0,1.0
27836,NaN,NaN,NaN,1.0,1.0,2.0


In [ ]:
# Групируем таблицу по закончившим обучение и сбросим индексы
number_of_users_df = tutorial_count_df.groupby('tutorial_finish')[['registration', 'purchase']].sum()
number_of_users_df = number_of_users_df.reset_index()
number_of_users_df

event_type,tutorial_finish,registration,purchase
0,1.0,8015.0,1143.0
1,2.0,1321.0,182.0
2,3.0,345.0,44.0
3,4.0,178.0,19.0
4,5.0,117.0,19.0
5,6.0,101.0,15.0
6,7.0,97.0,15.0
7,8.0,54.0,6.0
8,9.0,22.0,4.0


На данном этапе вводится метрика CR - Количеством совершенных целевых действие / Количество посетителей или пользователей.
Под Количеством совершенных целевых действие - я поринимаю кол. пользователей совершивших покупку.
Под Количество посетителей или пользователей - я принимаю кол. пользователей прошедших обучение один/несколько раз.
Получается:
CR - соотношение пользователей совершивших покупку от общего числа пользователей (прошедших обучение один/несколько раз)

In [ ]:
# Определяем новый признак CR в %
number_of_users_df['CR %'] = round(number_of_users_df['purchase']/number_of_users_df['registration']*100, 2)
number_of_users_df.head(5)


event_type,tutorial_finish,registration,purchase,CR %
0,1.0,8015.0,1143.0,14.26
1,2.0,1321.0,182.0,13.78
2,3.0,345.0,44.0,12.75
3,4.0,178.0,19.0,10.67
4,5.0,117.0,19.0,16.24


**Можно было бы объеденить малочисленные группы в одну но я не стал этого делать т.к. на графике видно что среди всех групп нет сильного разброса в значение CR

In [ ]:
# График 
fig_3 = px.bar(number_of_users_df,
              x= 'tutorial_finish',
              y= 'CR %',
              height=400,
              width=1000,
              labels={"value": "CR %", "index": "кол. пройденых обучений", "variable": "users совершие покупку"},
              title= 'CR по группам пользователей')

fig_3.show()

## <center> ВЫВОД

Процент пользователей совершивших покупку среди прошедших обучение один раз и пользователей прошедших обучение более одного раз существенных различий в CR не имеет, значит количество пройденых обучений никак не влияет на вероятность оплаты. Можно было бы выделить группу с 9 пройдеными обучениями, но она очень малочисленная (4 пользователя) чтобы по ней делать какие то выводы.

## <center> Гипотеза 3

<center> Определить, насколько прозрачен процесс взаимодействия с игрой.

In [ ]:
# Сводная таблица значение-время, столбцы-тип события, индекс-пользователи

"""
Берется максимальное время, чтобы получить максимально позднее обучение из всех начатых пользователем
(что бы увидеть пользователей начивших обучение после выбора сложности или 
вернувшихсы к обучению повторно после выбора сложности )
поскольку оно может покажет время обучения, произошедшее позже выбора уровня сложности
"""

events_pivot = events_df.pivot_table(
    values='start_time',index='user_id',columns='event_type',aggfunc='max') 

# Убираем лишнии столбцы и сбрасываем индекс
events_pivot = events_pivot[['tutorial_start','level_choice']].reset_index()

events_pivot.head(5)

event_type,user_id,tutorial_start,level_choice
0,27832,NaT,NaT
1,27833,2018-01-01 17:47:40,NaT
2,27834,2018-01-01 19:46:11,NaT
3,27835,2018-01-01 15:00:51,2018-01-01 20:37:22
4,27836,2018-01-01 15:40:43,NaT


In [ ]:
# Признак отражающий разницу времени между выбором уровня сложности и обучением
events_pivot['return_to_tutorial'] = events_pivot['level_choice'] - events_pivot['tutorial_start'] 

# Удаляю строки не содержащие значений
events_pivot = events_pivot.dropna(axis=0,subset='level_choice')

# избавляюсь от формата Timedelta
events_pivot['return_to_tutorial'] = events_pivot['return_to_tutorial']/ pd.Timedelta('1 hour')  

events_pivot.head(5)

event_type,user_id,tutorial_start,level_choice,return_to_tutorial
3,27835,2018-01-01 15:00:51,2018-01-01 20:37:22,5.608611
7,27839,2018-01-01 19:11:36,2018-01-01 22:37:50,3.437222
8,27840,2018-01-02 02:07:07,2018-01-02 05:18:42,3.193056
10,27842,2018-01-02 18:08:01,2018-01-02 08:46:03,-9.366111
11,27843,2018-01-02 13:34:51,2018-01-02 14:09:58,0.585278


In [ ]:
# маска для фильтрации отрицальных значений
mask_4 = events_pivot['return_to_tutorial'] < 0 

# Количество пользователей, перешедших на этап выбора уровня сложности level_choice
level_choice_count = events_pivot['user_id'].nunique() 

# Кол. вернувшихся на этап обучения
return_tutorial_count = events_pivot[mask_4]['user_id'].nunique() 

# Процент вернувшихся от достигших этапа выбора сложности
percent_returned_users = return_tutorial_count/level_choice_count

print('Процент вернувшиеся на этап обучения после выбора сложности '
      'составляет: {:.2%}'.format(percent_returned_users))

Процент вернувшиеся на этап обучения после выбора сложности составляет: 16.61%


## <center> ВЫВОД

Количество пользователей, дошедших до уровня выбора уровня сложности составляет: 8342 человека.  
Количество пользователей, начавших обучения после выбора уровня сложности составляет: 1386 человека.  
Только 17% пользователей начали обучение после выбора уровня сложности, что говорит об интуитивно понятном интерфейсе и правилах игры. Но 17% (1386 пользователей) это довольно много и к этой группе стоет присмотреться по внимательнее.

## <center> **ВЫВОД ПО ПРОЕКТУ**

Гипотеза 1 пользователи прошедшие обучение проходят этапы игры бастрее всех, помедленее это делают пользователи не закончившие обучение и медленее всех проходят этапы пользователи не проходившие обучение. Можно сделать вывод что обучение (даже не оконченое) сказывается на скорости прохождение этапов игры.  
Пользователи прошедшии обучение имеют наибольшую вероятность покупки 14% и их доля от всех покупателей подавляющая 90.4%  пользователи не закончившие обучение имеют вероятность покупки 8% их доля от всех покупателей 8.2% и не проходившие обучение имеют вероятность покупки менее 1% их доля от всех покупателей 1.4%  
При разборе гипотезы 2 выявлено что количество пройденых обучений не влияют на вероятность покупки т.к. у расматриваймых групп пользователи прошедшие обучение один раз и пользователи прошедшии обучение более одного раза существенных расхождений в вероятности покупки не выявлено.  
Гипотеза 3 выявила группу из 17% пользователей вернувшихся на этап обучения, эта группа нуждается в более подробном расмотрении и анализе: посмотреть их путь по всем этапам, узнать не отбращались он в службу поддержки (если она вообще у нас есть) и если да то с каким вопросом это поможет установить моменты в игре которые требуют доработки. Это большая по численности группа и выводы сделаные прие её подробном расмотрении могут улучшить продукт.

Из всего можно выделить что обучение очень позитивно сказывается на вовлеченности в игру (об этом говорит скорости прохождения этапов) и покупках совершенными пользователями прошедшими обучение 90.4% от всех покупок. Само обучение занимает не много времени в среднем 0.2 ч.. Необходими продвигать обучение размещая кнопку на более видном месте или предлага игровые поощрения за прохождение обучения.